In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.
!pip install clean-text
!pip install unicode
# !pip install keras==2.15.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.2/465.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.5 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=f09e45b49c52139075748d7fd293a8dcb296c0ed6ec624ada5f25926db30597f
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# améliore la rapidité d'entrainement
keras.mixed_precision.set_global_policy("mixed_float16")

In [4]:
import pandas as pd

df = pd.concat([
    pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv')[['OriginalTweet']],
    pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding='iso-8859-1')[['OriginalTweet']]
])
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  44955 non-null  object
dtypes: object(1)
memory usage: 351.3+ KB


# **Pré-traitement**

In [5]:
print(df['OriginalTweet'].isnull().sum())
df.dropna(subset=['OriginalTweet'], how='any', inplace=True)
df.reset_index(inplace=True, drop=True)

unique_elements = not df['OriginalTweet'].duplicated().any()

if unique_elements:
    print("Tous les éléments sont uniques.")
else:
    print("Certains éléments ne sont pas uniques.")

0
Tous les éléments sont uniques.


In [6]:
import cleantext
import re
# corrige les caractères unicode mal formaté
# converti les caracteres non ASCII en leur équivalent exemple : é devient e

df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: cleantext.clean(x,
                                                                         fix_unicode=True,
                                                                         to_ascii=True,
                                                                         lower=True,
                                                                         no_emails=True,
                                                                         no_urls=True,
                                                                         ))
df['OriginalTweet'] = df['OriginalTweet'].str.replace('<url>', '')
# traitement des character speciaux
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: re.sub(r'\W', ' ', x))
# on enleve les espaces en trop
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'\s{2,}', ' ', regex=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  44955 non-null  object
dtypes: object(1)
memory usage: 351.3+ KB


In [7]:
# initialisation pour l'utilisation du modèle GPT-2
import pandas as pd
# from tensorflow.keras.preprocessing.text import Tokenizer
from keras_nlp.models import GPT2CausalLMPreprocessor, GPT2CausalLM

preprocessor = GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)

gpt2_lm = GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=preprocessor
)

gpt2_lm.summary()


100%|██████████| 448/448 [00:00<00:00, 1.13MB/s]
100%|██████████| 446k/446k [00:00<00:00, 525kB/s]
100%|██████████| 0.99M/0.99M [00:01<00:00, 938kB/s]
100%|██████████| 484/484 [00:00<00:00, 488kB/s]
100%|██████████| 475M/475M [00:25<00:00, 19.6MB/s]


Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(df)

                                           OriginalTweet
0      trending  new yorkers encounter empty supermar...
1      when i couldn t find hand sanitizer at fred me...
2      find out how you can protect yourself and love...
3       panic buying hits  newyork city as anxious sh...
4       toiletpaper  dunnypaper  coronavirus  coronav...
...                                                  ...
44950  airline pilots offering to stock supermarket s...
44951  response to complaint not provided citing covi...
44952  you know it s getting tough when  kameronwilds...
44953  is it wrong that the smell of hand sanitizer i...
44954   tartiicat well new used rift s are going for ...

[44955 rows x 1 columns]


In [8]:
text_df = list(df['OriginalTweet'].astype(str).values)
print(text_df[:10])

['trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn sold out online grocers foodkick maxdelivery as coronavirus fearing shoppers stock up ', 'when i couldn t find hand sanitizer at fred meyer i turned to amazon but 114 97 for a 2 pack of purell check out how coronavirus concerns are driving up prices ', 'find out how you can protect yourself and loved ones from coronavirus ', ' panic buying hits newyork city as anxious shoppers stock up on food medical supplies after healthcare worker in her 30s becomes bigapple 1st confirmed coronavirus patient or a bloomberg staged event qanon qanon2018 qanon2020 election2020 cdc ', ' toiletpaper dunnypaper coronavirus coronavirusaustralia coronavirusupdate covid_19 9news corvid19 7newsmelb dunnypapergate costco one week everyone buying baby milk powder the next everyone buying up toilet paper ', 'do you remember the last time you paid 2 99 a gallon for regular gas in los angeles prices at the pump are going down a

In [9]:
import tensorflow as tf


with tf.device('/device:GPU:0'):
    num_epochs = 1
    # Peut bien fonctionner lorsque vous vous attendez à ce que le taux d'apprentissage diminue rapidement au début de l'entraînement.
    # Précision allant jusqu'à 35% avec un epoch à 1
    learning_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=len(text_df) * num_epochs,
    decay_rate=0.96,
    staircase=True
    )
    # On utilisera le paramètre ci-dessous lorsque nos données ne seront pas traiter
    # Permet un ajustement plus fin du taux d'apprentissage au fil du temps.
    # Peut mieux s'adapter à des schémas d'apprentissage moins réguliers (les #, http etc les trucs inutiles / non logique de nos données).
    # à utiliser avec un epoch au minimum de 3, 10 serait optimal

    # initial_learning_rate peut être mis à 1e-3
    # learning_rate = keras.optimizers.schedules.PolynomialDecay(
    #     initial_learning_rate=5e-5,
    #     decay_steps=len(text_df) * num_epochs,
    #     end_learning_rate=0.0,
    # )

    # on va crée un callbacks dans le cas ou l'entrainement devient régréssant
    # changer la valeur de patience selon l'epochs qu'on mettra
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
    )

    gpt2_lm.fit(
        x=text_df,
        epochs=num_epochs,
        batch_size = 32,
        callbacks=[early_stopping]
    )

1405/1405 ━━━━━━━━━━━━━━━━━━━━ 1012s 598ms/step - accuracy: 0.2881 - loss: 1.3880


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [ ]:
# model gpt-2 de base
for i in range(5):
# Ne pas laisser d'espace avant le " ça un impacte énorme !
  seed_text = "the coronavirus is a"
  # on peu rajouter le paramètre max_length si on veut
  generated_text = gpt2_lm.generate(seed_text, max_length=100)
  print("\nGPT-2 output:")
  print(generated_text)
  print('___________________________________________________________')


GPT-2 output:
the coronavirus is a  pandemic  it s not just the consumer who is at risk  it s also the business community  the economy  consumer  economy is at stake  
___________________________________________________________

GPT-2 output:
the coronavirus is a                            the                                     
___________________________________________________________

GPT-2 output:
the coronavirus is a  global  pandemic  but what is the impact on  consumer spending and consumer confidence  the report reveals that                                 
___________________________________________________________

GPT-2 output:
the coronavirus is a  socialdistancing virus  but how do you make it work when people walk in the grocery store     socialdistancing  socialdistancing  covid2019
___________________________________________________________

GPT-2 output:
the coronavirus is a  socialdistancing   socialdistancing    the  coronavirus has created a  shortage of food   a

# **Interface Gradio**

### Dépendances

In [10]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 12.5 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl

In [50]:
import gradio as gr
from datetime import datetime

# Définir votre thème personnalisé
theme_chat = gr.themes.Soft(
    primary_hue="teal",
    secondary_hue="cyan",
    neutral_hue="slate",
)

custom_css="""
@import url('https://fonts.googleapis.com/css2?family=Tangerine:wght@700&display=swap');

body {
    margin: 0;
    padding: 0;
}

.container {
    width: 80%;
    margin: 0 auto;
}

h1 {
    font-size: 3em;
    margin-top: 20px;
    font-family: 'Tangerine', serif;
}

p {
    font-size: 1.5em;
    color: turquoise;
    text-align: center;
}

.textbox {
    width: 80%;
    margin: 20px auto;
    padding: 10px;
    font-size: 1.2em;
}

.chatbox {
    border: 1px solid #ccc;
    padding: 20px;
    margin-top: 20px;
    border-radius: 8px;
}

.btn {
    display: inline-block;
    padding: 10px 20px;
    font-size: 1em;
    text-align: center;
    text-decoration: none;
    border-radius: 5px;
    cursor: pointer;
}

"""


def chat_bot_response(message, history):
    message = message.replace("?","").strip()
    # le paramètre max_length est à enlever pour l'instant
    return gpt2_lm.generate(message)

# Créer l'interface Gradio avec le thème personnalisé
interface = gr.ChatInterface(
    chat_bot_response,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Posez moi une question sur le CoronaVirus", container=False, scale=7),
    title="Vit'IA",
    description="Posez une question à Vit'IA :",
    examples=["The covid is a ?", "Symptoms of the covid are ?", "The best way to protect yourself from covid is to ?"],
    cache_examples=False,
    retry_btn="Générer une nouvelle réponse",
    undo_btn="Supprimer la réponse précédente",
    clear_btn="Effacer",
    submit_btn="Envoyer",
    css= custom_css,
    theme=theme_chat
)

# Lancer l'interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://52a467305442aa9f0b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
